In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
import soccerdata as sd

In [ ]:
from socceraction.data.opta import OptaLoader
from socceraction.data.statsbomb import StatsBombLoader

In [ ]:
from pathlib import PosixPath


ws = sd.WhoScored(
    #  "GER-Bundesliga", "GER-Bundesliga2", "ENG-Premier League", "ESP-La Liga", "FRA-Ligue 1", "ITA-Serie A"
    leagues=["GER-Bundesliga2"],
    seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=True,
)

In [ ]:
corona_games = [
    1376730,
    1376707,
    1376732,
    1376726,
    1376733,
    1376725,
    1376729,
    1376716,
    1376735,
    1376734,
    1376721,
    1376722,
    1376723,
    1376728,
    1376724,
    1376731,
    1376727,
    1376720,
    1376718,
    1376717,
    1376719,
]

loader = ws.read_events(output_fmt="loader")
# loader = ws.read_events(output_fmt='loader', skip=corona_games)

In [ ]:
# Now use this loader to load the data
game_list = []
# print("Games:")
for season in ["2122"]:
    df_games = loader.games(competition_id="GER-Bundesliga2", season_id=season)
    game_list.append(df_games)

# display(df_games.head())
# print(df_games.head())

In [ ]:
game_list[0].head()

In [ ]:
# tmp = pd.concat(game_list)
# df_all_games = pd.concat([tmp, df_all_games])

In [ ]:
df_all_games = game_list[0]

In [ ]:
df_all_games.head()

In [ ]:
df_all_games.shape

In [ ]:
import pandas as pd
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat
from tqdm import tqdm

In [ ]:
spadl.actiontypes_df()

In [ ]:
loader.events(df_all_games.game_id.values[0])

In [ ]:
spadl.opta.convert_to_actions(
    loader.events(df_all_games.game_id.values[0]), df_all_games.home_team_id.values[0]
)

In [ ]:
# 2. Convert direction of play
actions_ltr = pd.concat(
    [
        spadl.play_left_to_right(
            spadl.opta.convert_to_actions(
                loader.events(game[1]["game_id"]).dropna(subset=["player_id"]),
                game[1]["home_team_id"],
            ),
            game[1]["home_team_id"],
        )
        for game in df_all_games.iterrows()
    ]
)

In [ ]:
actions_ltr

In [ ]:
# 3. Train xT model
xTModel = xthreat.ExpectedThreat(l=32, w=24)
xTModel.fit(actions_ltr)

In [ ]:
import matplotsoccer as mps

# Inspect the learned heatmap
mps.heatmap(xTModel.xT, cmap="hot", linecolor="white", cbar="True")

# Inspect the interpolated heatmap used for predictions
interp = xTModel.interpolator()
x = np.linspace(0, 105, 1050)
y = np.linspace(0, 68, 680)
mps.heatmap(interp(x, y), cmap="hot", linecolor="white", cbar="True")

In [ ]:
xTModel.save_model("/home/morten/Develop/packing-report/xT-impact/models/xT_full_data")